In [ ]:
# !pip install ultralytics==8.3.15

In [ ]:
import os
import subprocess
from pathlib import Path

END_WITH_LOCAL = 'characters-and-dialouges-association-in-comics'

os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

BASE_DIR = os.getcwd()
print(f"BASE_DIR: {BASE_DIR}")

# Simple validation
if not (BASE_DIR.endswith('/content') or BASE_DIR.endswith(END_WITH_LOCAL)):
    raise ValueError(f"Expected to be in .../{END_WITH_LOCAL} or .../content directory, but got: {BASE_DIR}")

In [ ]:
import os
import cv2
import numpy as np
import json
from tqdm import tqdm
from ultralytics import YOLO
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils

model = YOLO('yolov8x-seg.pt')

In [ ]:
IMG_ROOT = "data/Manga109_released_2023_12_07/images"
JSON_ROOT = "data/MangaSegmentation/jsons"

output_dir = '/output'
os.makedirs(output_dir, exist_ok=True)

def get_pred_masks(result):
    """Trích xuất mask segmentation từ YOLOv8 result"""
    pred_masks = []
    if result.masks is not None:
        for mask, cls in zip(result.masks.data, result.boxes.cls):
            if int(cls) == 5 or result.names[int(cls)].lower() == 'bubble':
                pred_masks.append(mask.cpu().numpy())
    return pred_masks


def get_gt_masks(json_path, img_id):
    coco = COCO(json_path)
    ann_ids = coco.getAnnIds(imgIds=[img_id])
    anns = coco.loadAnns(ann_ids)
    gt_masks = []
    for ann in anns:
        if ann['category_id'] == 5:  # bubble id
            rle = maskUtils.frPyObjects(ann['segmentation'], ann['height'], ann['width'])
            gt_masks.append(maskUtils.decode(rle))
    return gt_masks

def compute_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / union if union > 0 else 0

ious, precisions, recalls, f1s = [], [], [], []

manga_list = sorted(os.listdir(IMG_ROOT))

for manga_name in tqdm(manga_list):
    manga_dir = os.path.join(IMG_ROOT, manga_name)
    json_path = os.path.join(JSON_ROOT, f"{manga_name}.json")

    if not os.path.exists(json_path):
        print(f"⚠️ Không tìm thấy JSON cho {manga_name}")
        continue

    # Chạy inference YOLOv8 cho toàn bộ folder truyện
    results = model.predict(
        source=manga_dir,
        save=False,
        conf=0.25,
        iou=0.5,
        imgsz=640,
        verbose=False
    )

    gt_masks = get_gt_masks(json_path)
    if not gt_masks:
        continue

    for res in results:
        pred_masks = get_pred_masks(res)
        if not pred_masks:
            continue

        # So sánh từng mask với GT
        for pm in pred_masks:
            best_iou = max(compute_iou(pm, gm) for gm in gt_masks)
            ious.append(best_iou)
            precisions.append(1 if best_iou > 0.5 else 0)
            recalls.append(1 if best_iou > 0.5 else 0)
            f1s.append(2 * precisions[-1] * recalls[-1] / (precisions[-1] + recalls[-1]) if (precisions[-1] + recalls[-1]) > 0 else 0)

In [ ]:
print("\n🎯 Kết quả segmentation bubble:")
print(f"👉 IoU trung bình:  {np.mean(ious):.3f}")
print(f"👉 Precision:       {np.mean(precisions):.3f}")
print(f"👉 Recall:          {np.mean(recalls):.3f}")
print(f"👉 F1-score:        {np.mean(f1s):.3f}")

In [ ]:
import matplotlib.pyplot as plt

sample = results[0]
img = cv2.imread(sample.path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

for mask in get_pred_masks(sample):
    img[mask > 0.5] = [255, 0, 0]  # tô đỏ vùng bubble

plt.imshow(img)
plt.axis('off')
plt.title('Predicted Bubbles')
plt.show()
